This notebook takes in the paper as written in `paper.md` and outputs an interactive version `paper_interactive.md`. By interactive, we mean a version where 
- static .png figure files are replaced with the interactive (holoviews/bokeh-powered) .html versions, and
- additional hyperlinks to various data files that support each figure (machine-readable data, for instance).

In [ ]:
from os.path import exists
import os

In [7]:
def load_file(fname='paper.md'):
    with open(fname) as f:
        p = f.read()
    return p

def write_paper(string, fname='test.txt'):
    with open(fname, 'w') as f:
        f.write(string)

paper = load_file()

figs = [s for s in paper.split('\n') if s.startswith('![')]

newcaps = load_file('interactive_captions.md')

Filename handling

In [8]:
def extract_from_between(string, start, end):
    """
    Extract substring from string delineated by start and end.
    """
    try:
        return string.split(start)[1].split(end)[0]
    except:
        return ''

def get_ref(fig):
    return extract_from_between(fig, '){', '}').split(' ')[0]
    
def get_filename(fig):
    return extract_from_between(fig, '](', '){')

def get_caption(fig):
    return extract_from_between(fig, '![', '](')

def get_new_ext(fname, ext, new_ext):
    return fname.replace(ext, new_ext)

Apparently, we need to render each html figure to find their height in the final markdown/html document.

In [9]:
from selenium import webdriver
driver = webdriver.PhantomJS()

def get_max_html_height(fname_html):
    driver.get('file://'+os.getcwd()+'/'+fname_html)
    return max([e.size['height'] for e in driver.find_elements_by_class_name('bk')])

/Users/doppler/anaconda3/envs/nitrateflux/lib/python3.7/site-packages/selenium/webdriver/phantomjs/webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnings.warn('Selenium support for PhantomJS has been deprecated, please use headless '


Update figure captions

In [10]:
def add_interactive_figure(fig, paper):
    fname = get_filename(fig)
    cap = get_caption(fig)
    ref = get_ref(fig)
    ext = fname.split('.')[-1]

    fname_html = get_new_ext(fname, ext, 'html')
    if exists(fname_html):
        paper = paper.replace(fname, fname_html)
        
        height_html = get_max_html_height(fname_html)
        new_ref = ref.split(' ')[0] + f' width=100% height={height_html}'
        paper = paper.replace(ref, new_ref)

    replace_caption = False
    for new_ext in ['csv', 'nc', 'hvz']:
        new_cap = cap + f" Machine-readable formats of this figure: "
        if exists(get_new_ext(fname, ext, new_ext)):
            replace_caption = True
            new_cap = new_cap + f"[{get_new_ext(fname, ext, new_ext)}.split('/')[-1]]({get_new_ext(fname, ext, new_ext)})"
    if replace_caption:
        paper = paper.replace(cap, new_cap)
        
    return paper

    

for fig in figs:
    paper = add_interactive_figure(fig, paper)

Finally, replace figures if there is a predefined new caption in the file [interactive_captions.md](interactive_captions.md) for their interactive version (Figures identified by their reference tags).

In [13]:
figs = [s for s in paper.split('\n') if s.startswith('![')]
new_figs = [s for s in newcaps.split('\n') if s.startswith('![')]

# paper = paper.replace('.png)', '.html)')

for o, n in [
    (old, new)
    for old in figs for new in new_figs
    if get_ref(old)==get_ref(new)
]:
    paper = paper.replace(o, n)

In [14]:
write_paper(paper, 'paper_interactive.md')